<a href="https://colab.research.google.com/github/gmb408/OSSP-KBO-prediction/blob/version-0.1/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 분석 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
result = pd.read_csv("/content/statiz_pre.csv", index_col=0)

In [ ]:
# 단타 feature 생성
result['1타'] = result['안타'] - result['2타'] - result['3타'] - result['홈런']

In [ ]:
# 사용 features 명시
infos = ['ID','이름','생일','팀','시즌','포지션','나이']
stats = ['G','타수','득점','안타','1타','2타','3타','홈런','루타','타점','도루','도실','볼넷','사구','고4','삼진'\
        ,'병살','희타','희비','타율','출루','장타','OPS','wOBA','wRC+','WAR+']

In [ ]:
# 해당 시즌 이전까지의 누적 기록
def get_cumulative(player_id, season, stat=stats ,df=result):
    try:
        result_cum = df[(df['ID']==player_id) & (df['시즌'] < season)][stat]
        result_cum = result_cum.mean()
        
    except:
        result_cum = [np.nan for i in stat]
        result_cum = pd.Series(result_cum)
    
    return result_cum

In [ ]:
# 모든 stats에 대한 누적 stat
stats_cum = [i + '_누적' for i in stats]

In [ ]:
# 누적 stat 데이터프레임에 반영
result[stats_cum] = result.apply(lambda x: get_cumulative(x['ID'], x['시즌']), axis=1)

In [ ]:
# lag stat (직전 몇년의 기록)
def get_lag(player_id, season, lag, stat=stats, df=result):
    try:
        result_lag = df[(df['ID']==player_id)&(df['시즌'] == season-lag)][stat].values[0]
        result_lag = pd.Series(result_lag)
        
    except:
        result_lag = [np.nan for i in stat]
        result_lag = pd.Series(result_lag)
        
    return result_lag


In [ ]:
# 직전 1,2,3년의 기록
stats_lag1 = [i + '_LAG1' for i in stats]
stats_lag2 = [i + '_LAG2' for i in stats]
stats_lag3 = [i + '_LAG3' for i in stats]

In [ ]:
# lag1
result[stats_lag1] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=1), axis=1)

In [ ]:
# lag2
result[stats_lag2] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=2), axis=1)

In [ ]:
# lag3
result[stats_lag3] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=3), axis=1)

In [ ]:
# 내년 기록 (OPS, 장타, 출루)
# 데이터 프레임의 정답 셋 만들기 위함

def get_next_year(player_id, season, stat=['장타','출루','OPS'],df=result):
    try:
        result = df[(df['ID']==player_id)&(df['시즌']==season+1)][stat].values[0]
        result = pd.Series(result)
    except:
        result = [np.nan for i in stat]
        result = pd.Series(result)
        
    return result

In [ ]:
# 장타, 출루, OPS의 내년 기록
stats_next = [i + '_NEXT' for i in ['장타', '출루', 'OPS']]
result[stats_next] = result.apply(lambda x: get_next_year(x['ID'], x['시즌']), axis=1)

In [ ]:
# 변환 데이터 저장
result.to_csv("/content/statiz_Final.csv")
result

,ID,이름,생일,팀,시즌,포지션,나이,G,타석,타수,...,타율_LAG3,출루_LAG3,장타_LAG3,OPS_LAG3,wOBA_LAG3,wRC+_LAG3,WAR+_LAG3,장타_NEXT,출루_NEXT,OPS_NEXT
0,0,이종범,1970-08-15,해,1994,SS,-0.972904,1.164774,561.0,1.820499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586,0.397,0.983
1,1,테임즈,1986-11-10,N,2015,1B,0.177011,1.722593,595.0,1.629513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.679,0.427,1.106
2,2,심정수,1975-05-05,현,2003,RF,-0.052972,1.443684,601.0,1.544630,...,0.304,0.400,0.551,0.951,0.413,149.9,5.31,0.480,0.385,0.864
3,0,이종범,1970-08-15,해,1997,SS,-0.282955,1.195764,577.0,1.714395,...,0.393,0.452,0.581,1.033,0.462,198.3,11.77,NaN,NaN,NaN
4,0,이종범,1970-08-15,해,1996,SS,-0.512938,0.823885,525.0,1.466822,...,0.280,0.331,0.432,0.762,0.350,127.4,6.14,0.581,0.428,1.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5791,2192,임종찬,2001-09-28,한,2021,RF,-1.892835,-1.376400,146.0,-0.775491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321,0.228,0.549
5792,1845,박찬호,1995-06-05,K,2020,SS,-0.742921,1.691603,530.0,1.679028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.313,0.331,0.644
5793,1998,정보근,1999-08-31,롯,2020,C,-1.662853,-0.043833,152.0,-0.768418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5794,1808,나종덕,1998-03-16,롯,2019,C,-1.662853,0.544976,209.0,-0.400594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
